In [ ]:
import subprocess
from datetime import datetime
from pathlib import Path

import fsspec
from utils import sync_notebook

In [ ]:
fs = fsspec.filesystem("file")
current_date = datetime.today().strftime("%Y-%m-%d")
output_folder = f"results/{current_date}/"
files = fs.glob(f"{output_folder}memray-*.json")
fs.rm(files)
fs.mkdirs(output_folder, exist_ok=True)

In [ ]:
dataset = "gpm_imerg"

In [ ]:
def run_memray(file: str):
    output_file = output_folder + "memray-" + dataset + "-" + Path(file).stem + ".bin"
    summary_file = output_folder + "memray-" + dataset + "-" + Path(file).stem + ".json"
    command = [
        "memray",
        "run",
        "--force",
        "--output",
        output_file,
        file,
        "--dataset",
        dataset,
    ]
    subprocess.run(command)
    summary_command = [
        "memray",
        "stats",
        "--force",
        "--json",
        "--output",
        summary_file,
        output_file,
    ]
    subprocess.run(summary_command)

In [ ]:
input_methods = ["resample-rioxarray-h5netcdf", "resample-odc", "resample-xesmf"]

In [ ]:
notebooks = []
for fp in input_methods:
    notebooks.extend(fs.glob(f"{fp}*.ipynb"))

In [ ]:
for file in notebooks:
    sync_notebook(file)

In [ ]:
modules = []
for fp in input_methods:
    modules.extend(fs.glob(f"{fp}*.py"))
for file in modules:
    run_memray(file)